# Retención de clientes

Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Necesitamos predecir si un cliente dejará el banco pronto. Se tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.

Crearemos un modelo que prediga cuando un cliente potencialmente podría abandonar el banco.

## Carga de datos y librerías

In [1]:
# Cargar todas las librerías
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split #Librería para poder dividir los dataframes
from sklearn.tree import DecisionTreeClassifier #Librería para el modelado por árbol de decisión
from sklearn.ensemble import RandomForestClassifier #Librería para el modelado por bosque aleatorio
from sklearn.linear_model import LogisticRegression #Librería para el modelado por regresión logística
from sklearn.preprocessing import StandardScaler #Librería para hacer el escalado de características
#Librerías para calcular, la exactitud, valor de 1 y la curva precision-recall
from sklearn.metrics import accuracy_score, f1_score, precision_recall_curve, roc_curve, r2_score
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.utils import shuffle

### Descripción de los datos

**Características**

- RowNumber: índice de cadena de datos
- CustomerId: identificador de cliente único
- Surname: apellido
- CreditScore: valor de crédito
- Geography: país de residencia
- Gender: sexo
- Age: edad
- Tenure: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
- Balance: saldo de la cuenta
- NumOfProducts: número de productos bancarios utilizados por el cliente
- HasCrCard: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
- IsActiveMember: actividad del cliente (1 - sí; 0 - no)
- EstimatedSalary: salario estimado

**Objetivo**

- Exited: El cliente se ha ido (1 - sí; 0 - no)

In [2]:
# Carga los archivos de datos 
df = pd.read_csv('Churn.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Observamos que el dataframe no presenta problema con los tipos de datos, parece ser que todos son adecuados. Pero se observa que la columna tenure presenta valores ausentes, más adelante determinaremos que realizar con estos valores ausentes.

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


## Análisis de datos

In [4]:
na_ratio = ((df.isnull().sum() / len(df))*100).sort_values(ascending = False)
na_ratio

Tenure             9.09
RowNumber          0.00
CustomerId         0.00
Surname            0.00
CreditScore        0.00
Geography          0.00
Gender             0.00
Age                0.00
Balance            0.00
NumOfProducts      0.00
HasCrCard          0.00
IsActiveMember     0.00
EstimatedSalary    0.00
Exited             0.00
dtype: float64

Observamos que el porcentaje de valores ausentes de la columna tenure es del 9.09%, no es un valor elevado, pero no podemos eliminar esos valores, ya que reduciríamos la cantidad de valores para los conjuntos de entrenamiento y validación y no se observa una relación con otra columna, por tanto, se obtará por sustituir los valores con el promedio de los valores de tenure.

In [5]:
df['Tenure'] = df['Tenure'].fillna(value=df['Tenure'].mean())

A continuacion, convertimos los valores de tenure a entero.

In [6]:
df['Tenure'] = df['Tenure'].astype(int)

In [7]:
df.duplicated().sum()

0

El dataframe no presenta valores duplicados.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


Observamos que el dataframe no presenta valores ausentes ni duplicados, por tanto podemos proseguir con el desarrollo de nuestros modelos.

## Equilibrio de clases

In [9]:
df['Exited'].value_counts(normalize=True)

0    0.7963
1    0.2037
Name: Exited, dtype: float64

Observamos que casi el 80% de las respuestas son cero y alrededor del 20% son 1, por tanto, concluimos que el dataframe presenta desequilibrio de clases, problema que se atenderá y resolverá en las siguientes secciones.

## Entrenamiento y evaluación de los modelos

### Preparación de datos

#### Segmentación de datos

Antes de dividir el dataframe en conjunto de entrenamiento y validación, debemos transformar las características categóricas en numéricas, para lo que, primeramente removeremos las columnas Surname, RowNumber y CustomerId que no nos aportan datos útiles para nuestros modelo y posteriormente transformaremos las características categóricas usando codificación One-Hot. No se utilizará el argumento drop_first para mantener ambos valores en Gender (Male y Female) y en Geography (France, Spain y Germany).

In [10]:
df_ohe = df.drop(['Surname', 'RowNumber', 'CustomerId'], axis=1)
df_ohe = pd.get_dummies(df_ohe)
df_ohe

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,1,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,1,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,1,0,0,1,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1,0,0,1


Obtenemos las características, que serían todas la columnas menos exited y el objetivo, que sería únicamente la columna Exited.

In [11]:
features = df_ohe.drop('Exited', axis=1)
target = df_ohe['Exited']

Dividimos el dataframe en conjunto de entrenamiento y validación en los siguientes porcentajes:

- 75% Entrenamiento.
- 25% Validación

In [12]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
print('Porcentaje del conjunto de entrenamiento:',100*features_train.shape[0]/df_ohe.shape[0])
print('Porcentaje del conjunto de validación:',100*features_valid.shape[0]/df_ohe.shape[0])

Porcentaje del conjunto de entrenamiento: 75.0
Porcentaje del conjunto de validación: 25.0


#### Escalado de características

Antes de entrenar el modelo, nuestro dataframe cuenta con características con diferentes escalas, que serían las columnas:

- CreditScore
- Age
- Tenure
- Balance
- NumOfProducts
- EstimatedSalary

In [13]:
#Codigo utilizado para silenciar la alerta de python al escalar las características
pd.options.mode.chained_assignment = None

In [14]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
226,0.442805,-0.841274,1.471429,-1.224577,0.817772,1,1,-1.269750,1,0,0,1,0
7756,-0.310897,-0.270730,0.748652,0.641783,-0.896874,1,1,0.960396,0,0,1,1,0
2065,-0.259274,-0.556002,1.110041,-1.224577,0.817772,1,0,0.661864,1,0,0,0,1
2800,1.217157,1.155631,1.471429,1.290462,0.817772,1,0,-1.039476,1,0,0,0,1
7028,0.690598,-1.221637,-0.335513,1.142121,-0.896874,0,0,-0.851729,0,1,0,0,1


### Regresión logística

#### Entrenamiento del modelo

In [15]:
model = DecisionTreeClassifier(random_state=12345)
#model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

#### Exactitud

In [16]:
accuracy_valid = accuracy_score(target_valid, predicted_valid)
accuracy_valid

0.7788

Observamos que el modelo tiene una buena exactitud, a continuación lo compararemos contra la observación de puros ceros y veremos que pasa.

In [17]:
target_pred_constant = pd.Series(0, index=target.index)
print(accuracy_score(target, target_pred_constant)) 

0.7963


El modelo presenta casi la misma exactitud, lo cual confirma lo que veíamos el la sección Equilibrio de clases, el dataframe presenta un desequilibrio de clases. 

Calcularemos el valor de f1 y AUC-ROC

#### F1 y AUC-ROC

In [18]:
f1 = f1_score(target_valid, predicted_valid)
f1

0.4931255728689276

In [19]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_zero_valid = probabilities_valid[:, 0]
auc_roc = roc_auc_score(target_valid, probabilities_zero_valid)

print(auc_roc)

0.32162612066300444


In [24]:
model = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
#model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))

F1: 0.4948064211520302


#### Sobremuestreo

In [26]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled


features_upsampled, target_upsampled = upsample(features_train, target_train, 50)

model = DecisionTreeClassifier(random_state=12345)
#model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))

F1: 0.4711830131445905


#### Submuestreo

In [22]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)]+ [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])

    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)

    return features_downsampled, target_downsampled


features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)

model = DecisionTreeClassifier(random_state=12345)
#model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))

F1: 0.4626068376068376


## Conclusiones

Al realizar un submuestreo o sobremuestreo se sigue obteniendo un valor inferior de f1 al buscado (0.59), sin importar si se usa el modelo de árbol de decisión o regresión logística, así que se envía el proyecto para verificar si hay algún error que se haya realizado en el código que afectara el desempeño del modelo.